In [1]:
import joblib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Carpark

In [2]:
def train_model(X, y, model=LinearRegression()):
    
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    # scale
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # fit
    model.fit(X_train_scaled, y_train)
    
    # score/predict
    score_training = model.score(X_train_scaled, y_train)
    score_testing = model.score(X_test_scaled, y_test)
    preds = model.predict(X_test)
    
    return X_test_scaled, y_test, score_training, score_testing, preds, model, scaler

In [3]:
models = [LinearRegression(), Ridge(), Lasso(), ElasticNet()]

In [4]:
df_carpark = pd.read_csv('carpark.csv')
df_carpark

,SystemCodeNumber,Capacity,Occupancy,LastUpdated
0,BHMBCCMKT01,577,61,2016-10-04 07:59:42
1,BHMBCCMKT01,577,64,2016-10-04 08:25:42
2,BHMBCCMKT01,577,80,2016-10-04 08:59:42
3,BHMBCCMKT01,577,107,2016-10-04 09:32:46
4,BHMBCCMKT01,577,150,2016-10-04 09:59:48
...,...,...,...,...
35712,Shopping,1920,1517,2016-12-19 14:30:33
35713,Shopping,1920,1487,2016-12-19 15:03:34
35714,Shopping,1920,1432,2016-12-19 15:29:33
35715,Shopping,1920,1321,2016-12-19 16:03:35


In [5]:
df_carpark['hour'] = [x.hour for x in pd.to_datetime(df_carpark['LastUpdated'])]

In [6]:
df_carpark['weekday'] = [x.weekday() for x in pd.to_datetime(df_carpark['LastUpdated'])]

In [7]:
X_car = df_carpark[['hour', 'Capacity', 'weekday']]
y_car = df_carpark['Occupancy']

In [8]:
results = []
for model in models:
    
    X_test_scaled, y_test, score_training, score_testing, preds, model, scaler = train_model(X_car, y_car, model)
    results.append(
        {'X_test_scaled':X_test_scaled, 
         'y_test':y_test,
         'score_training':score_training,
         'score_testing':score_testing,
         'preds':preds, 
         'model':model,
         'scaler': scaler}
    )

In [9]:
for result in results:
    print(result['model'], '\ntesting score: ', result['score_testing'], '\n---')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 
testing score:  0.6491165149706263 
---
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001) 
testing score:  0.6491168782362673 
---
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False) 
testing score:  0.6490757148895164 
---
ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False) 
testing score:  0.5785336355309201 
---


In [10]:
X_car.values[1]

array([  8, 577,   1])

In [11]:
def get_pred(hour, capacity, weekday, model, scaler):
    
    X = [[hour,capacity, weekday]]
    X_scaled = scaler.transform(X)
    pred = model.predict(X_scaled)
    
    return(int(pred[0]))
    

In [12]:
get_pred(16, 577, 4, results[0]['model'], results[0]['scaler'])

466

#### Joblib
You can use the Joblib library for storing models, scalers, etc.

In [13]:
joblib.dump(results[0]['scaler'], "stdscaler.gz")

['stdscaler.gz']

In [14]:
new_scaler_var = joblib.load('stdscaler.gz')

In [15]:
new_scaler_var

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
joblib.dump(results[0]['model'], 'linearmodel.gz')

['linearmodel.gz']

In [17]:
new_model_var = joblib.load('linearmodel.gz')

In [18]:
myvar = 'Stephen Curry from WAY DOWNTOWN!!!'

In [19]:
joblib.dump(myvar, 'steph.file')

['steph.file']

In [20]:
joblib.load('steph.file')

'Stephen Curry from WAY DOWNTOWN!!!'